In [1]:
suppressMessages(library("rwwa"))

In [11]:
gmst <- read.table("gmst.txt", col.names = c("year", "gmst"))

In [28]:
df <- read.csv("tx5x-monthly_era5_cities.csv")

df$year <- as.integer(substr(df$time,1,4))
df$month <- as.integer(substr(df$time,6,7))

In [20]:
for(cnm in colnames(df)[-1][1]) {
    df_c <- merge(gmst, df[,c("year", "month", cnm)])
    
    df_jun <- df_c[df_c$month == 6,]
    df_jul <- df_c[df_c$month == 7,]
    
    mdl_jun <- fit_ns("gev", "shift", df_jun, varnm = cnm, covnm = "gmst", lower = F)
    mdl_jul <- fit_ns("gev", "shift", df_jul, varnm = cnm, covnm = "gmst", lower = F)
    
    # bootstrap whole years to get difference in trends
    set.seed(42)
    boot_res <- sapply(1:5, function(i) {
        ysamp <- sample(1:nrow(df_c[df_c$month == 6,]), replace = T)
        boot_df <- mdl$data[sample(1:nrow(mdl$data), replace = T), 
            ]
        tryCatch({
            boot_mdl <- refit(mdl, new_data = boot_df)
            mdl_ests(boot_mdl, cov_f, cov_cf, ev = ev, rp = rp)
        }, error = function(cond) {
            return(rep(NA, length(obs_res)))
        })
    })
}

In [31]:
        ysamp <- sample(1:nrow(df_c[df_c$month == 6,]), replace = T)
